In [2]:
import yfinance as yf
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from IPython.display import display

# Function to fetch and process stock data
def fetch_ticker_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data['5d_avg'] = data['Close'].rolling(window=5).mean()
    data['10d_avg'] = data['Close'].rolling(window=10).mean()
    data['Volume_Change'] = data['Volume'].pct_change()
    data['Target'] = data['Close'].shift(-1)  # Tomorrow's price
    data = data.dropna()  # Remove NaN values
    return data

# Function to train SVR model and make predictions
def predict_stock_price(ticker, start_date, end_date):
    data = fetch_ticker_data(ticker, start_date, end_date)
    features = ['5d_avg', '10d_avg', 'Volume_Change']
    X = data[features]
    y = data['Target']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train Support Vector Regression model
    model = SVR(kernel='rbf')
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)

    # Get the last prediction (next day's predicted price)
    last_prediction = model.predict([X.iloc[-1]])[0]

    return last_prediction, mse

# Function to handle button click and display results in the notebook
def on_predict_button_click(ticker, start_date, end_date):
    try:
        # Predict stock price change
        predicted_price, mse = predict_stock_price(ticker, start_date, end_date)
        result_label.value = f"Predicted Next Day Price: ${predicted_price:.2f}"
        mse_label.value = f"Mean Squared Error: {mse:.2f}"
    except Exception as e:
        result_label.value = f"Error: {e}"

# Create widgets for user input
ticker_input = widgets.Text(value='', placeholder='Enter Stock Ticker', description='Ticker:', disabled=False)
start_date_input = widgets.DatePicker(description='Start Date:')
end_date_input = widgets.DatePicker(description='End Date:')
predict_button = widgets.Button(description="Predict")
result_label = widgets.Label(value="Predicted Next Day Price: $")
mse_label = widgets.Label(value="Mean Squared Error: ")

# Set up button click event
predict_button.on_click(lambda x: on_predict_button_click(ticker_input.value, start_date_input.value, end_date_input.value))

# Display the widgets in the notebook
display(ticker_input, start_date_input, end_date_input, predict_button, result_label, mse_label)

Text(value='', description='Ticker:', placeholder='Enter Stock Ticker')

DatePicker(value=None, description='Start Date:')

DatePicker(value=None, description='End Date:')

Button(description='Predict', style=ButtonStyle())

Label(value='Predicted Next Day Price: $')

Label(value='Mean Squared Error: ')

[*********************100%***********************]  1 of 1 completed
